In [10]:
import moviepy.editor as mp
import os

def extract_video_segment(video_path, start_time, end_time=None, fps=None, save_path="output.mov"):
    # Check if the input file is .mp4
    filename, file_extension = os.path.splitext(video_path)
    if file_extension == ".mp4":
        # Convert .mp4 to .mov
        clip = mp.VideoFileClip(video_path)
        video_path = filename + ".mov"
        clip.write_videofile(video_path, codec="libx264", audio_codec="aac")

    video = mp.VideoFileClip(video_path)
    if end_time is None:
        end_time = video.duration
    segment = video.subclip(start_time, end_time)
    if fps is None:
        fps = video.fps
    segment.write_videofile(save_path, codec="libx264", audio_codec="aac", fps=fps)

video_dir = "./analysis/labeling/start_2023-11-16T19-50-24.725+00-00/2024-01-26T16-39+00-00_label/"
video_path = video_dir + "original_binary_output.mp4"
save_path = video_path.replace("original_binary_output.mp4", "binary_output_120s.mov")
extract_video_segment(video_path, start_time=120, save_path=save_path)

t:   1%|▏         | 14/960 [2:28:38<167:24:10, 637.05s/it, now=None]

Moviepy - Building video ./analysis/labeling/start_2023-11-16T19-50-24.725+00-00/2024-01-26T16-39+00-00_label/original_binary_output.mov.
Moviepy - Writing video ./analysis/labeling/start_2023-11-16T19-50-24.725+00-00/2024-01-26T16-39+00-00_label/original_binary_output.mov



t:   1%|▏         | 14/960 [2:28:44<167:30:30, 637.45s/it, now=None]

Moviepy - Done !
Moviepy - video ready ./analysis/labeling/start_2023-11-16T19-50-24.725+00-00/2024-01-26T16-39+00-00_label/original_binary_output.mov


t:   1%|▏         | 14/960 [2:28:44<167:30:43, 637.47s/it, now=None]

Moviepy - Building video ./analysis/labeling/start_2023-11-16T19-50-24.725+00-00/2024-01-26T16-39+00-00_label/binary_output_120s.mov.
Moviepy - Writing video ./analysis/labeling/start_2023-11-16T19-50-24.725+00-00/2024-01-26T16-39+00-00_label/binary_output_120s.mov



t:   1%|▏         | 14/960 [2:28:49<167:36:40, 637.84s/it, now=None]

Moviepy - Done !
Moviepy - video ready ./analysis/labeling/start_2023-11-16T19-50-24.725+00-00/2024-01-26T16-39+00-00_label/binary_output_120s.mov


In [16]:
import glob
import yaml

import yaml
from algorithm.FishDetector import FishDetector
from algorithm.InputOutputHandler import InputOutputHandler

video_dirs = glob.glob("./analysis/labeling/*")
yaml_file = './analysis/demo/demo_settings.yaml'

for video_dir in video_dirs:
    video_dir = video_dir.replace("\\", "/")
    input_file = video_dir + "/original.mp4"
    with open(yaml_file) as f:
        settings_dict = yaml.load(f, Loader=yaml.SafeLoader)
        if input_file is not None:
            settings_dict["input_file"] = input_file
            settings_dict['output_directory'] = video_dir
            print("replacing input file with: ", input_file)

    input_output_handler = InputOutputHandler(settings_dict)
    detector = FishDetector(settings_dict)
    object_history = {}

    while input_output_handler.get_new_frame():
        if float(input_output_handler.frame_no) / 2 % 1 != 0:
            continue
        detections, processed_frame_dict, runtimes = detector.detect_objects(
            input_output_handler.current_raw_frame
        )
        object_history = detector.associate_detections(detections, object_history)
        input_output_handler.handle_output(
            processed_frame_dict, object_history, runtimes, detector=detector
        )

    if input_output_handler.output_csv_name is not None:
        detections = input_output_handler.get_detections_pd(object_history)
        detections = detector.classify_detections(detections)
        detections.to_csv(input_output_handler.output_csv_name, index=False)
        
    # cut off the first 120 seconds as burn-in and convert to mov for roboflow
    record_processing_frame = settings_dict["record_processing_frame"]
    out_dir = next(d for d in os.listdir(video_dir) if os.path.isdir(os.path.join(video_dir, d)))
    base_name = os.path.basename(video_dir)

    out_video_path = os.path.join(video_dir, out_dir, f"original_{record_processing_frame}_output.mp4")
    save_binary_path = os.path.join(video_dir, out_dir, f"{base_name}_{record_processing_frame}_output_120s.mov")
    extract_video_segment(out_video_path, start_time=120, save_path=save_binary_path)
    print(f"Saved {save_binary_path}")

replacing input file with:  ./analysis/labeling/start_2023-05-09T13-31-14.149+00-00/original.mp4
Processed 0.2 % of video. Runtimes [ms]: getFrame: 5 | Enhance: 3 | DetectTrack: 0 | Total: 17 | FPS: 64.5
Processed 0.3 % of video. Runtimes [ms]: getFrame: 4 | Enhance: 5 | DetectTrack: 0 | Total: 14 | FPS: 65.1
Processed 0.5 % of video. Runtimes [ms]: getFrame: 10 | Enhance: 9 | DetectTrack: 0 | Total: 29 | FPS: 59.4
Processed 0.7 % of video. Runtimes [ms]: getFrame: 3 | Enhance: 7 | DetectTrack: 0 | Total: 16 | FPS: 57.6
Processed 0.8 % of video. Runtimes [ms]: getFrame: 4 | Enhance: 4 | DetectTrack: 0 | Total: 13 | FPS: 59.2
Processed 1.0 % of video. Runtimes [ms]: getFrame: 3 | Enhance: 3 | DetectTrack: 0 | Total: 12 | FPS: 60.2
Processed 1.2 % of video. Runtimes [ms]: getFrame: 4 | Enhance: 4 | DetectTrack: 0 | Total: 14 | FPS: 60.7
Processed 1.3 % of video. Runtimes [ms]: getFrame: 4 | Enhance: 4 | DetectTrack: 0 | Total: 13 | FPS: 60.0
Processed 1.5 % of video. Runtimes [ms]: getFr

Moviepy - Done !
Moviepy - video ready ./analysis/labeling/start_2023-05-09T13-31-14.149+00-00\2024-01-29T12-46+00-00_label\original_binary_output.mov
Moviepy - Building video ./analysis/labeling/start_2023-05-09T13-31-14.149+00-00\2024-01-29T12-46+00-00_label\start_2023-05-09T13-31-14.149+00-00_binary_output_120s.mov.
Moviepy - Writing video ./analysis/labeling/start_2023-05-09T13-31-14.149+00-00\2024-01-29T12-46+00-00_label\start_2023-05-09T13-31-14.149+00-00_binary_output_120s.mov



Moviepy - Done !
Moviepy - video ready ./analysis/labeling/start_2023-05-09T13-31-14.149+00-00\2024-01-29T12-46+00-00_label\start_2023-05-09T13-31-14.149+00-00_binary_output_120s.mov
Saved ./analysis/labeling/start_2023-05-09T13-31-14.149+00-00\2024-01-29T12-46+00-00_label\start_2023-05-09T13-31-14.149+00-00_binary_output_120s.mov
replacing input file with:  ./analysis/labeling/start_2023-11-06T18-16-49.060+00-00/original.mp4
Processed 0.2 % of video. Runtimes [ms]: getFrame: 6 | Enhance: 4 | DetectTrack: 0 | Total: 16 | FPS: 52.9
Processed 0.3 % of video. Runtimes [ms]: getFrame: 4 | Enhance: 4 | DetectTrack: 0 | Total: 17 | FPS: 52.9
Processed 0.5 % of video. Runtimes [ms]: getFrame: 4 | Enhance: 3 | DetectTrack: 0 | Total: 17 | FPS: 53.3
Processed 0.7 % of video. Runtimes [ms]: getFrame: 9 | Enhance: 5 | DetectTrack: 0 | Total: 22 | FPS: 51.6
Processed 0.8 % of video. Runtimes [ms]: getFrame: 7 | Enhance: 5 | DetectTrack: 0 | Total: 17 | FPS: 52.1
Processed 1.0 % of video. Runtimes 

Moviepy - Done !
Moviepy - video ready ./analysis/labeling/start_2023-11-06T18-16-49.060+00-00\2024-01-26T17-46+00-00_label\original_binary_output.mov
Moviepy - Building video ./analysis/labeling/start_2023-11-06T18-16-49.060+00-00\2024-01-26T17-46+00-00_label\start_2023-11-06T18-16-49.060+00-00_binary_output_120s.mov.
Moviepy - Writing video ./analysis/labeling/start_2023-11-06T18-16-49.060+00-00\2024-01-26T17-46+00-00_label\start_2023-11-06T18-16-49.060+00-00_binary_output_120s.mov



Moviepy - Done !
Moviepy - video ready ./analysis/labeling/start_2023-11-06T18-16-49.060+00-00\2024-01-26T17-46+00-00_label\start_2023-11-06T18-16-49.060+00-00_binary_output_120s.mov
Saved ./analysis/labeling/start_2023-11-06T18-16-49.060+00-00\2024-01-26T17-46+00-00_label\start_2023-11-06T18-16-49.060+00-00_binary_output_120s.mov
replacing input file with:  ./analysis/labeling/start_2023-11-16T19-50-24.725+00-00/original.mp4
Processed 0.2 % of video. Runtimes [ms]: getFrame: 10 | Enhance: 6 | DetectTrack: 0 | Total: 25 | FPS: 54.6
Processed 0.3 % of video. Runtimes [ms]: getFrame: 5 | Enhance: 6 | DetectTrack: 0 | Total: 16 | FPS: 54.2
Processed 0.5 % of video. Runtimes [ms]: getFrame: 5 | Enhance: 6 | DetectTrack: 0 | Total: 17 | FPS: 55.8
Processed 0.7 % of video. Runtimes [ms]: getFrame: 6 | Enhance: 5 | DetectTrack: 0 | Total: 18 | FPS: 56.4
Processed 0.8 % of video. Runtimes [ms]: getFrame: 4 | Enhance: 6 | DetectTrack: 0 | Total: 19 | FPS: 56.6
Processed 1.0 % of video. Runtimes

...

Moviepy - Done !
Moviepy - video ready ./analysis/labeling/start_2023-11-18T18-23-01.832+00-00\2024-01-26T18-00+00-00_label\start_2023-11-18T18-23-01.832+00-00_binary_output_120s.mov
Saved ./analysis/labeling/start_2023-11-18T18-23-01.832+00-00\2024-01-26T18-00+00-00_label\start_2023-11-18T18-23-01.832+00-00_binary_output_120s.mov
replacing input file with:  ./analysis/labeling/start_2023-11-19T17-14-18.851+00-00/original.mp4
Processed 0.2 % of video. Runtimes [ms]: getFrame: 10 | Enhance: 11 | DetectTrack: 0 | Total: 39 | FPS: 16.0
Processed 0.3 % of video. Runtimes [ms]: getFrame: 14 | Enhance: 29 | DetectTrack: 0 | Total: 76 | FPS: 16.6
Processed 0.5 % of video. Runtimes [ms]: getFrame: 14 | Enhance: 16 | DetectTrack: 0 | Total: 44 | FPS: 17.8
Processed 0.7 % of video. Runtimes [ms]: getFrame: 14 | Enhance: 14 | DetectTrack: 0 | Total: 44 | FPS: 18.6
Processed 0.8 % of video. Runtimes [ms]: getFrame: 20 | Enhance: 20 | DetectTrack: 0 | Total: 54 | FPS: 18.9
Processed 1.0 % of video.

Moviepy - Done !
Moviepy - video ready ./analysis/labeling/start_2023-11-19T17-14-18.851+00-00\2024-01-26T18-06+00-00_label\original_binary_output.mov
Moviepy - Building video ./analysis/labeling/start_2023-11-19T17-14-18.851+00-00\2024-01-26T18-06+00-00_label\start_2023-11-19T17-14-18.851+00-00_binary_output_120s.mov.
Moviepy - Writing video ./analysis/labeling/start_2023-11-19T17-14-18.851+00-00\2024-01-26T18-06+00-00_label\start_2023-11-19T17-14-18.851+00-00_binary_output_120s.mov



Moviepy - Done !
Moviepy - video ready ./analysis/labeling/start_2023-11-19T17-14-18.851+00-00\2024-01-26T18-06+00-00_label\start_2023-11-19T17-14-18.851+00-00_binary_output_120s.mov
Saved ./analysis/labeling/start_2023-11-19T17-14-18.851+00-00\2024-01-26T18-06+00-00_label\start_2023-11-19T17-14-18.851+00-00_binary_output_120s.mov
replacing input file with:  ./analysis/labeling/start_2023-11-20T17-35-40.863+00-00/original.mp4
Processed 0.2 % of video. Runtimes [ms]: getFrame: 13 | Enhance: 14 | DetectTrack: 0 | Total: 44 | FPS: 24.6
Processed 0.3 % of video. Runtimes [ms]: getFrame: 14 | Enhance: 17 | DetectTrack: 0 | Total: 48 | FPS: 22.3
Processed 0.5 % of video. Runtimes [ms]: getFrame: 15 | Enhance: 16 | DetectTrack: 0 | Total: 49 | FPS: 21.9
Processed 0.7 % of video. Runtimes [ms]: getFrame: 16 | Enhance: 15 | DetectTrack: 0 | Total: 47 | FPS: 21.7
Processed 0.8 % of video. Runtimes [ms]: getFrame: 15 | Enhance: 17 | DetectTrack: 0 | Total: 51 | FPS: 21.6
Processed 1.0 % of video.

Moviepy - Done !
Moviepy - video ready ./analysis/labeling/start_2023-11-20T17-35-40.863+00-00\2024-01-26T18-11+00-00_label\original_binary_output.mov
Moviepy - Building video ./analysis/labeling/start_2023-11-20T17-35-40.863+00-00\2024-01-26T18-11+00-00_label\start_2023-11-20T17-35-40.863+00-00_binary_output_120s.mov.
Moviepy - Writing video ./analysis/labeling/start_2023-11-20T17-35-40.863+00-00\2024-01-26T18-11+00-00_label\start_2023-11-20T17-35-40.863+00-00_binary_output_120s.mov



Moviepy - Done !
Moviepy - video ready ./analysis/labeling/start_2023-11-20T17-35-40.863+00-00\2024-01-26T18-11+00-00_label\start_2023-11-20T17-35-40.863+00-00_binary_output_120s.mov
Saved ./analysis/labeling/start_2023-11-20T17-35-40.863+00-00\2024-01-26T18-11+00-00_label\start_2023-11-20T17-35-40.863+00-00_binary_output_120s.mov
